# CVPR2021 NAS竞赛Track 1第2名方案
(队伍名称：ANS - 榜单名次：2 - 队长姓名：陆顺)

## 1. 代码及结果复现：

### 1.1 环境安装

In [ ]:
cd ./cvpr21_nas_track1/Track1_Submit
bash shell/setup.sh

### 1.2 提交时所使用的checkpoint （对应榜单最好结果）

最好模型对应路径：cd ./cvpr21_nas_track1/Track1_Submit/files/supernet.th

### 1.3 基于supernet生成的最终提交结果（对应榜单最好结果）

In [ ]:
cd ./cvpr21_nas_track1/Track1_Submit
bash ./shell/test.sh

关于这一任务的训练log输出，我们记录在./train_eval/eval/test.log

### 1.4 复现supernet的训练过程（对应榜单最好结果）

In [ ]:
cd ./cvpr21_nas_track1/Track1_Submit
bash shell/train.sh

关于这一任务的训练log输出，我们记录在./train_eval/train/train.log

## 2. 方法说明与技术讨论
我们的技术方案以单路径采样为主，并使用多种不同的技巧在此基础上进一步提升。以下将进行详细讨论，所有的训练技巧我们通过train_supernet.py一个文件全部实现。

### 2.1 数据预处理

在实验中我们发现，使用和官方提供的代码中相同的数据预处理并不好，根据官方代码得到的数据预处理如下：

In [ ]:
train_transform = transforms.Compose([
        transforms.RandomCrop(32, 4),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1)]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5071, 0.4865, 0.4409], std=[0.1942, 0.1918, 0.1958])
    ])

我们发现，去掉ColorJitter并添加Cutout的效果更好，代码如下

In [ ]:
train_transform = transforms.Compose([
        transforms.RandomCrop(32, 4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        Cutout(16),
        transforms.Normalize(mean=[0.5071, 0.4865, 0.4409], std=[0.1942, 0.1918, 0.1958])
    ])

### 2.2 模型调优

#### 2.2.1 超网模型一致性构建

官方提供的单模型构建repo中，在相邻两层且stride=1时，若通道不同则包含1x1卷积，若通道相同，则不包含1x1卷积；


实验中，我们发现，无论相邻两层通道是否相同，均采用1x1卷积，可以提升最终的一致性

#### 2.2.2 BN参数设置

官方提供的单模型构建repo中，BN使用默认配置，其中track_running_stats=True。
实验中，我们发现，当设置BN参数为False时，最终的排序一致性更好。
因为每次采样一条路径，统计方差会很大，所以设置track_running_stats=False更合理。

#### 2.2.3 少量模型进一步调优

超网训练结束后，部分模型存在训练不充分的情况，我们从50000个模型中均匀采样了200个模型，对这一局部进行继续训练（具体采样的结构详见./Track1_Submit/files/Track1_200_archs.json)，我们发现继续训练30个epoch可使得超网的预测一致性进一步提升。
实验中，我们测试了5/10/15/20/25/30/40/50/60/70/80epoch，分别使用不同的checkpoint对50000个模型预测后，提交发现使用30epoch最好。
发现这一方法有效后，我们继续使用随机采样的方式，但由于时间紧急我们只测试了采样200个/100个模型，继续微调，发现采样100个模型继续微调性能还会提升，这100个模型对应的模型文件为./Track1_Submit/files/Track1_100_archs.json。

#### 2.2.4 蒸馏方法的应用

根据BigNAS的启发，我们使用超网中的最大模型的输出作为软标签，来蒸馏采样模型。同时训练采样模型和最大模型。这里的采样模型和2.2.3中的采样模型相同，还是只对200个模型进行采样。代码实现如下，实现过程中，由于时间紧急，只微调了两次。第一次微调，使用初始学习率1e-3微调30epoch；第二次微调，使用初始学习率4e-3微调30epoch。发现这样可以获得一致性的进一步提升。

In [ ]:
output = model(input_var)  # compute output
loss = criterion(output, target_var)  # compute loss
if args.distill:
    teacher_output = model(input_var, [16, 16, 16, 16, 16, 16, 16, 32, 32, 32, 32, 32, 32, 64, 64, 64, 64, 64, 64, 64])
    teacher_loss = criterion(teacher_output, target_var)
    soft_target_var = torch.nn.functional.softmax(teacher_output, dim=1).detach()
    distill_loss = soft_criterion(output, soft_target_var)
    loss = 0.5 * (loss + teacher_loss) + 2.0 * distill_loss

### 2.3 参数设置及遍历

#### 2.3.1 每一步采样模型数量

受到FairNAS启发，每一步采样时，累积4/6/8/10/12/16个模型的梯度一并更新，发现使用6时，超网排序性最好

#### 2.3.2 超网训练的epoch数量

超网的训练epoch，我们尝试了100/125/150/200/230/240/250/260/300，发现使用250epoch训练超网最好

#### 2.3.3 超网训练的学习率

超网训练的学习率我们尝试了0.1/0.05/0.025/0.01/0.005，发现使用0.025最好

#### 2.3.4 超网训练的weight decay

超网训练的学习率我们尝试了1e-4/2e-4/3e-4/4e-4/5e-4/6e-4，发现使用5e-4最好